In [21]:
import json

import pandas as pd
import requests
from bs4 import BeautifulSoup


def get_page(url):
    s = requests.session()
    r = s.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    return soup


def get_a_movie_info(url: str) -> pd.DataFrame:
    '''
    Takes in a movie URL like "https://letterboxd.com/film/goon/"
    return the Dataframe of the movie's info
    '''
    soup = get_page(url)
    i = 0
    begin = 0
    end = 0
    for item in str(soup).split('\n'):
        if ("* <![CDATA[ */" in item):
            begin = i
        if ("/* ]]> */" in item):
            end = i + 1
        i += 1
    info_list = [str(soup).split('\n')[begin:end], url]
    movie_df = pd.json_normalize(json.loads(info_list[0][1]))
    return movie_df


def post_a_movie_info(url):
    '''
    actually posts movie info to our endpoint
    '''
    df = get_a_movie_info(url)
    post_df = df[["image", "director", "dateModified", "productionCompany", "releasedEvent", "url",
                  "actors", "dateCreated", "name", "aggregateRating.reviewCount",
                  "aggregateRating.ratingValue", "aggregateRating.ratingCount"]]
    post_df = post_df.rename(columns={"aggregateRating.reviewCount": "reviewCount",
                                      "aggregateRating.ratingValue": "ratingValue",
                                      "aggregateRating.ratingCount": "ratingCount"})
    print(post_df)
    post_data = post_df.to_dict('records')[0]
    for k in post_data.keys():
        post_data[k] = str(post_data[k])
    for k in post_data.keys():
        print(k)
        print(type(post_data[k]))
    print(post_data)
    r = requests.post("http://127.0.0.1:8000/endpoint/movie_table_post/", data=post_data)
    print(r)
    print(r.content)
    return


# columns:
# Index(['image', '@type', 'director', 'dateModified', 'productionCompany',
#        'releasedEvent', '@context', 'url', 'actors', 'dateCreated', 'name',
#        'genre', '@id', 'countryOfOrigin', 'aggregateRating.bestRating',
#        'aggregateRating.reviewCount', 'aggregateRating.@type',
#        'aggregateRating.ratingValue', 'aggregateRating.description',
#        'aggregateRating.ratingCount', 'aggregateRating.worstRating'],
#       dtype='object')


In [22]:
post_a_movie_info("https://letterboxd.com/film/goon/")

                                               image   
0  https://a.ltrbxd.com/resized/film-poster/6/1/9...  \

                                            director dateModified   
0  [{'@type': 'Person', 'name': 'Michael Dowse', ...   2023-07-09  \

                                   productionCompany   
0  [{'@type': 'Organization', 'name': 'Inferno Pi...  \

                                       releasedEvent   
0  [{'@type': 'PublicationEvent', 'startDate': '2...  \

                                 url   
0  https://letterboxd.com/film/goon/  \

                                              actors dateCreated  name   
0  [{'@type': 'Person', 'name': 'Seann William Sc...  2011-10-03  Goon  \

   reviewCount  ratingValue  ratingCount  
0         4280         3.26        36271  
image
<class 'str'>
director
<class 'str'>
dateModified
<class 'str'>
productionCompany
<class 'str'>
releasedEvent
<class 'str'>
url
<class 'str'>
actors
<class 'str'>
dateCreated
<class 'str'>
name
<class